# Speech Commands Dataset

In [1]:
import os

from lib.scDataset import SpeechCommandsDataset

dataset_path = '/root/data/speech_commands'
full_dataset = SpeechCommandsDataset(root_path=dataset_path, mode='full', include_rate=True)
train_dataset = SpeechCommandsDataset(root_path=dataset_path, mode='train', include_rate=False)
test_dataset = SpeechCommandsDataset(root_path=dataset_path, mode='test', include_rate=False)
val_dataset = SpeechCommandsDataset(root_path=dataset_path, mode='validation', include_rate=False)

len(full_dataset), len(train_dataset), len(test_dataset), len(val_dataset)

(64721, 51088, 6835, 6798)

In [2]:
sample_rates = set()
labels = set()
audio_shape = set()
for audio, label, sample_rate in full_dataset:
    sample_rates.add(sample_rate)
    labels.add(label)
    audio_shape.add(audio.shape)

sample_rates, len(labels)

({16000}, 30)

In [3]:
audio_shape

{torch.Size([1, 5945]),
 torch.Size([1, 6144]),
 torch.Size([1, 6688]),
 torch.Size([1, 6827]),
 torch.Size([1, 7339]),
 torch.Size([1, 7431]),
 torch.Size([1, 7510]),
 torch.Size([1, 7851]),
 torch.Size([1, 8022]),
 torch.Size([1, 8174]),
 torch.Size([1, 8192]),
 torch.Size([1, 8363]),
 torch.Size([1, 8534]),
 torch.Size([1, 8875]),
 torch.Size([1, 8917]),
 torch.Size([1, 9046]),
 torch.Size([1, 9387]),
 torch.Size([1, 9558]),
 torch.Size([1, 9660]),
 torch.Size([1, 9728]),
 torch.Size([1, 9899]),
 torch.Size([1, 10032]),
 torch.Size([1, 10070]),
 torch.Size([1, 10240]),
 torch.Size([1, 10403]),
 torch.Size([1, 10411]),
 torch.Size([1, 10582]),
 torch.Size([1, 10752]),
 torch.Size([1, 10923]),
 torch.Size([1, 11094]),
 torch.Size([1, 11146]),
 torch.Size([1, 11264]),
 torch.Size([1, 11435]),
 torch.Size([1, 11520]),
 torch.Size([1, 11606]),
 torch.Size([1, 11776]),
 torch.Size([1, 11889]),
 torch.Size([1, 11947]),
 torch.Size([1, 12118]),
 torch.Size([1, 12261]),
 torch.Size([1, 12288

In [2]:
from lib.scDataset import BackgroundNoise
noise_dataset = BackgroundNoise(root_path=dataset_path)
for noise_type, noise, sample_rate in noise_dataset:
    print(f'{noise_type}, {noise.shape}, {sample_rate}')

pink_noise, torch.Size([1, 960000]), 16000
running_tap, torch.Size([1, 978488]), 16000
exercise_bike, torch.Size([1, 980062]), 16000
doing_the_dishes, torch.Size([1, 1522930]), 16000
dude_miaowing, torch.Size([1, 988891]), 16000
white_noise, torch.Size([1, 960000]), 16000
